# Mathematical Techniques

### Importing Libraries and Configuration

We'll be using the htex configuration for Parsl. Read more [here.]( https://github.com/Parsl/parsl/blob/master/parsl/configs/htex_local.py)

In [1]:
import numpy as np

import parsl
import os
from parsl.app.app import python_app, bash_app
from parsl.configs.local_threads import config

parsl.load(config)

## Block Method of Matrix Multiplication

The block method of matrix multiplication between two matrices A and B of sizes sXq and rXs is as follows:

<img src="formula.png"
     style="float: left; size: 15px;" />

In [2]:
matrix_a = [[2,3,4,5],[4,5,6,7],[4,5,6,7]]
matrix_b = [[3,4],[4,5],[6,7],[8,9]]

In [3]:
@python_app
def multiply(inputs=[]):
    return inputs[0]*inputs[1]

In [4]:
def multiply_matrices(A,B):
    C = [[0 for _ in range(len(B[0]))] for _ in range(len(A))]
    
    for q in range(len(A)):
        for r in range(len(B[0])):
            
            s = len(A[0])
            
            elements = []
            for i in range(s):
                elements.append(multiply(inputs=[A[q][i],B[i][r]]))
           
            elements = [i.result() for i in elements]
            C[q][r] = sum(elements)
    
    return C

In [5]:
multiply_matrices(matrix_a, matrix_b)

[[82, 96], [124, 146], [124, 146]]

## The Mandelbrot Fractal Set

Code adapted from: https://www.geeksforgeeks.org/mandelbrot-fractal-set-visualization-in-python/

Let's run the Mandelbrot Fractal Set for a single pixel first:

In [8]:
@python_app
def mandelbrot(x, y):
    
    from numpy import complex, array 
    import colorsys
    
    def rgb_conversion(i): 
        color = 255 * array(colorsys.hsv_to_rgb(i / 255.0, 1.0, 0.5)) 
        return tuple(color.astype(int)) 
    
    c0 = complex(x, y)
    c = 0
    for i in range(1, 1000): 
        if abs(c) > 2:
            return rgb_conversion(i) 
        c = c * c + c0 
    return (0, 0, 0) 

In [9]:
mandelbrot(40,10).result()

(127, 6, 0)

Let us now execute the code using the Parsl function for an entire image

In [15]:
from PIL import Image 
from numpy import complex, array 
import colorsys 

WIDTH = 1024
img = Image.new('RGB', (WIDTH, int(WIDTH / 2)))
pixels_raw = [[0 for _ in range(img.size[1])] for _ in range(img.size[0])]

for x in range(img.size[0]):
    for y in range(img.size[1]):
        a = (x - (0.75 * WIDTH)) / (WIDTH / 4)
        b = (y - (WIDTH / 4)) / (WIDTH / 4)
        pixels_raw[x][y] = mandelbrot(a, b)

pixels_raw = [[i.result() for i in x] for x in pixels_raw]

In [22]:
pixels = img.load()

for x in range(img.size[0]):
    for y in range(img.size[1]):
        pixels[x,y] = pixels_raw[x][y]
        
img.show()

# Basic Genetic Algorithm

Code adopted from https://towardsdatascience.com/genetic-algorithm-implementation-in-python-5ab67bb124a6

In [7]:
import numpy

@python_app
def cal_pop_fitness(equation_inputs, pop):
    fitness = numpy.sum(pop*equation_inputs, axis=1)
    return fitness

@python_app
def select_mating_pool(pop, fitness, num_parents):
    parents = numpy.empty((num_parents, pop.shape[1]))
    for parent_num in range(num_parents):
        max_fitness_idx = numpy.where(fitness == numpy.max(fitness))
        max_fitness_idx = max_fitness_idx[0][0]
        parents[parent_num, :] = pop[max_fitness_idx, :]
        fitness[max_fitness_idx] = -99999999999
    return parents

@python_app
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    crossover_point = numpy.uint8(offspring_size[1]/2)

    for k in range(offspring_size[0]):
        parent1_idx = k%parents.shape[0]
        parent2_idx = (k+1)%parents.shape[0]
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

@python_app
def mutation(offspring_crossover):
    for idx in range(offspring_crossover.shape[0]):
        random_value = numpy.random.uniform(-1.0, 1.0, 1)
        offspring_crossover[idx, 4] = offspring_crossover[idx, 4] + random_value
    return offspring_crossover

In [9]:
import numpy
import ga

"""
The y=target is to maximize this equation ASAP:
    y = w1x1+w2x2+w3x3+w4x4+w5x5+6wx6
    where (x1,x2,x3,x4,x5,x6)=(4,-2,3.5,5,-11,-4.7)
    What are the best values for the 6 weights w1 to w6?
    We are going to use the genetic algorithm for the best possible values after a number of generations.
"""

equation_inputs = [4,-2,3.5,5,-11,-4.7]
num_weights = 6

"""
Genetic algorithm parameters:
    Mating pool size
    Population size
"""
sol_per_pop = 8
num_parents_mating = 4

pop_size = (sol_per_pop,num_weights) 
new_population = numpy.random.uniform(low=-4.0, high=4.0, size=pop_size)

num_generations = 5

for generation in range(num_generations):
    fitness = cal_pop_fitness(equation_inputs, new_population)
    parents = select_mating_pool(new_population, fitness, num_parents_mating).result()

    offspring_crossover = crossover(parents, offspring_size=(pop_size[0]-parents.shape[0], num_weights))
    offspring_mutation = mutation(offspring_crossover)

    new_population[0:parents.shape[0], :] = parents
    new_population[parents.shape[0]:, :] = offspring_mutation

TypeError: float() argument must be a string or a number, not 'AppFuture'

In [ ]:

fitness = cal_pop_fitness(equation_inputs, new_population)
best_match_idx = numpy.where(fitness == numpy.max(fitness))

print("Best solution : ", new_population[best_match_idx, :])
print("Best solution fitness : ", fitness[best_match_idx])